# Implanatação de algumas Bibliotacas

In [ ]:
import csv
import numpy as np
import pandas as pd
import sys
from tkinter import Tk
from tkinter import filedialog
from datetime import date, datetime, timedelta
#import os
#from ciso8601 import parse_datetime

# Criando variaveis para os arquivos permitidos

In [ ]:
p_cmpid7  = pd.read_excel('PQ382.xlsx')      ### leitura do arquivo com os sinais dos cpmids do quadro 382
param7    = pd.read_excel('LQ382.xlsx')      ### leitura do arquivo com o layout atualizado do quadro 382

In [ ]:
control  = 0                                              ### controle de backup    
p_cmpid7['cmpid'] = p_cmpid7.cmpid.astype(str)            ### transforma o campo cmpid - QE 382 - do arq auxiliar para string

In [ ]:
## FUNÇÃO PARA SELEÇÃO DO ARQUIVO

class qe_s:

    def __init__(self):
        self.df = {nome[s]: [] for s in range(nome.shape[0])}
  
    def make_df(self):
        lines = []
        root = Tk()
        root.attributes("-topmost", True)
        root.withdraw()
        for file in filedialog.askopenfilenames(filetypes=[("Arquivos de Text", "*.txt")]):
            for line in open(file, encoding="utf-8"):
                line = line.replace(",",".").strip()
                for x in range(nome.shape[0]):
                    self.df[nome[x]].append(line[i[x]:j[x]])
                    
        return pd.DataFrame(self.df)

In [ ]:
nome     = param7.Campo                      ### guarda os nomes das colunas existentes no quadro 380
i        = param7.Ind_inf                    ### guarda a posição no arquivo que inicia um determinado campo
j        = param7.Ind_sup                    ### guarda a posição no arquivo que termina um determinado campo

quadro = qe_s()       
df_main = quadro.make_df()
df_main

## SELECINE O(S) ARQUIVO(S) DESEJADO(S)

In [ ]:
###################################
###Importacao do Quadro382.xlsx ###
####que serão usados para fins#####
######### de comparação ###########
###################################

#Pasta ENTCODIGO
arq_entcod = pd.read_excel('Quadro382.xlsx','Cod_SUSEP',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta MRFMESANO
arq_mrfmesano = pd.read_excel('Quadro382.xlsx','MRFMESANO',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta Bib_DefCamposEstatísticos
arq_defcamesta = pd.read_excel('Quadro382.xlsx','Bib_DefCamposEstatísticos',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta rel_CMPID_TPMOID
arq_relacao = pd.read_excel('Quadro382.xlsx','rel_CMPID_TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

#Pasta TPMOID
arq_tpmoid = pd.read_excel('Quadro382.xlsx','TPMOID',fileEncoding = "UTF-8",encoding = 'latin_1', dtype=str)

In [ ]:
# Criando uma lista para guardar todas as criticas impeditivas
criticas_impeditivas = []
criticas = []

# Criticas (Validacoes)

In [ ]:
#7398.3 Verifica se o campo sequencial ESCSEQ é uma sequência válida, que se inicia em 0000001
array3 = df_main['ESCSEQ']
x = list(array3)                                     ##Dados do arquivo
arrayteste = x[:]
arrayteste.sort()                                    ##Dados sequenciados
if (arrayteste == x):                                ##Comparando
    print('Tudo certo, esta de forma sequencial.')
else: 
    print('ERRO!!!, Não prossiga.')
    critica7398_3 = str('Critica 7398.3, Não temos uma sequencia valida na coluna ESCSEQ')
    criticas_impeditivas.append(critica7398_3)

In [ ]:
#7398.2 Verifica o tamanho padrão da linha (188 caracteres) 
tamanho = 0
for i in range(0,df_main.shape[0]):                    #Passando por cada linha
    for iten in df_main.columns:                       #Passando por cada coluna
        tamanho = tamanho + len(df_main[iten][i])      #Contando Caracteres por linha
    if tamanho != 188:
        print('Erro na linha ',i,' o número de caracteres é de: ',tamanho)
        critica7398_2 = ("Critica 7398.2, Erro na linha ",i," o número de caracteres é de: ",tamanho)
        criticas_impeditivas.append(critica7398_2)
    tamanho = 0

In [ ]:
#7398.4 Verifica se o campo ENTCODIGO corresponde à sociedade que está enviando o FIP/SUSEP
entcod = set(arq_entcod['Cod_SUSEP'].astype('str'))                   #Escolhendo a coluna do arquivo modelo
dadosentcod = set(df_main['ENTCODIGO'])                               #Escolhendo a coluna dos dados
for itemtentcod in dadosentcod: 
    if itemtentcod not in entcod:
        print('ERRO!! ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        critica7398_4 = ('Critica 7398.4, ENTCODIGO não corresponde a uma operação valida', itemtentcod)
        criticas_impeditivas.append(critica7398_4)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtentcod)
        continue

In [ ]:
#7398.5 Verifica se o campo MRFMESANO corresponde, respectivamente, ao ano, mês e último dia do mês de referência do FIP/SUSEP 
df_main.loc[:,'DTMRFMESANO'] = df_main['MRFMESANO'].astype('datetime64')     #Criando uma nova coluna no formato Date
mrfmesano = set(arq_mrfmesano['DATAS'].astype('datetime64'))                 #Escolhendo a coluna do arquivo modelo
dadosmrfmesano = set(df_main['DTMRFMESANO'])                                 #Escolhendo a coluna dos dados
for itemmrfmesano in dadosmrfmesano: 
    if itemmrfmesano not in mrfmesano:
        print('ERRO!! MRFMESANO não corresponde a uma operação valida', itemmrfmesano)
        critica7398_5 = ('Critica 7398.5, MRFMESANO não corresponde a uma Data valida', itemmrfmesano)
        criticas_impeditivas.append(critica7398_5)
    else:
        print('MRFMESANO corresponde a uma Data válida.', itemmrfmesano)
        continue

In [ ]:
#7398.6 Verifica se o campo QUAID corresponde ao quadro 382
array6 = set(df_main['QUAID'])           ##Verificando todos os Quadros que estamos tratando no documento.
for itemarray6 in array6:
    if itemarray6 == '382':
        print('Este se trata do Quadro:',itemarray6)
    else:
        print('ERRO!! Este se trata do Quadro:',itemarray6)
        critica7398_6 = ('Critica 7398.6, Este esta se tratando do(s) Quadro(s):',itemarray6)
        criticas_impeditivas.append(critica7398_6)

In [ ]:
#7398.7 Verifica se o código do tipo de movimento é valido (conforme tabela ‘Tipos Movimentos’ do FIPSUSEP) 
tpmoid = set(arq_tpmoid['TPMOID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
dadostpmoid = set(df_main['TPMOID'])                                 #Escolhendo a coluna dos dados
for itemtpmoid in dadostpmoid: 
    if itemtpmoid not in tpmoid:
        print('ERRO!! O código do tipo de movimento não é valido (conforme tabela ‘Tipos Movimentos’ do FIPSUSEP) .', itemtpmoid)
        critica7398_7 = ('Critica 7398.7, O código do tipo de movimento não é valido (conforme tabela ‘Tipos Movimentos’ do FIPSUSEP) .', itemtpmoid)
        criticas.append(critica7398_7)
    else:
        print('O código do tipo de movimento é valido (conforme tabela ‘Tipos Movimentos’ do FIPSUSEP) .', itemtpmoid)
        continue

In [ ]:
#7398.8 Verifica se o campo CMPID corresponde a um tipo de operação válida (conforme tabela ‘Bib_DefCamposEstatisticos’ do FIPSUSEP) 
defcamesta = set(arq_defcamesta['CMPID'].astype('str'))             #Escolhendo a coluna do arquivo modelo
dadoscmpid = set(df_main['CMPID'])                                 #Escolhendo a coluna dos dados
for itemcmpid in dadoscmpid: 
    if itemcmpid not in defcamesta:
        print('ERRO!! O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        critica7398_8 = ('Critica 7398.8, O campo CMPID não corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        criticas.append(critica7398_8)
    else:
        print('O campo CMPID corresponde a um tipo de operação válida (conforme tabela Bib_DefCamposEstatísticos).', itemcmpid)
        continue

In [ ]:
#7398.9 Verifica se o PLNCODIGO pertence à tabela ‘Planos’ da entidade.

In [ ]:
##########################################
##Readicionando as colunas str para DATE##
##########################################

df_main.loc[:,'DTESCDATAINICIORO'] = df_main['ESCDATAINICIORO'].astype('datetime64')
df_main.loc[:,'DTESCDATAFIMRO'] = df_main['ESCDATAFIMRO'].astype('datetime64')
df_main.loc[:,'DTESCDATAEMISSRO'] = df_main['ESCDATAEMISSRO'].astype('datetime64')
df_main.loc[:,'DTESCDATAINICIORD'] = df_main['ESCDATAINICIORD'].astype('datetime64')
df_main.loc[:,'DTESCDATAFIMRD'] = df_main['ESCDATAFIMRD'].astype('datetime64')
df_main.loc[:,'DTESCDATAEMISSRD'] = df_main['ESCDATAEMISSRD'].astype('datetime64')

In [ ]:
#7398.10 Verifica se os campos ESCVALORMOVRO, ESCVALORMOVRD, ESCIMPSEG, ESCVALORCARO, ESCVALORCARD, ESCVALORCIRO e ESCVALORCIRD são float.

###########################################
##Readicionando as colunas str para float##
###########################################

df_main.loc[:,'fESCVALORMOVRO'] = df_main['ESCVALORMOVRO'].astype('float64')
df_main.loc[:,'fESCVALORMOVRD'] = df_main['ESCVALORMOVRD'].astype('float64')
df_main.loc[:,'fESCIMPSEG'] = df_main['ESCIMPSEG'].astype('float64')
df_main.loc[:,'fESCVALORCARO'] = df_main['ESCVALORCARO'].astype('float64')
df_main.loc[:,'fESCVALORCARD'] = df_main['ESCVALORCARD'].astype('float64')
df_main.loc[:,'fESCVALORCIRO'] = df_main['ESCVALORCIRO'].astype('float64')
df_main.loc[:,'fESCVALORCIRD'] = df_main['ESCVALORCIRD'].astype('float64')

#Adicionado coluna ESCVALORMOVRO, ESCVALORMOVRD, ESCIMPSEG, ESCVALORCARO, ESCVALORCARD, ESCVALORCIRO e ESCVALORCIRD no tipo Float

In [ ]:
#7398.13 Valida a correspondência entre os campos TPMOID e CMPID
mod_relacao = set(arq_relacao['relacao'].astype('str'))
relacao = [df_main['CMPID'] + df_main['TPMOID']]
for itemrelacao in relacao:
    relacao2 = set(itemrelacao)
    for itemrelacao2 in relacao2:
        if itemrelacao2 not in mod_relacao:
            print('ERRO!! A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            critica7398_13 = ('Critica 7398.13, A relação ',itemrelacao2,' (CMPID/TPMOID) não é uma relação possivel.')
            criticas_impeditivas.append(critica7398_13)
        else:
            print('A relação ',itemrelacao2,' (CMPID/TPMOID) é uma relação possivel.')
        continue

In [ ]:
criticas_impeditivas  #Lista das criticas impeditivas

In [ ]:
df_criticas_i = pd.DataFrame(criticas_impeditivas)     #criacao do DataFrame das criticas impeditivas
df_criticas_i = df_criticas_i.rename(columns={0:'Descrição'})
df_criticas_i

In [ ]:
if criticas_impeditivas == []:                             # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica Impeditiva encontrada no Quadro 382')
else:
    print('Foi encontrada uma ou mais Criticas Impeditivas no Quadro 382')
    df_criticas_i.to_csv('Criticas_Impeditivas_382.csv')   # Gerando csv das criticas impeditivas

# Criticas (Validacoes) Não Impeditivas

In [ ]:
#7398.11 Verifica se o campo ESCCODCESS corresponde a um código de sociedade válido
dadosesrcodcess = set(df_main['ESCCODCESS'])       #Escolhendo a coluna dos dados
for itemtesrcodcess in dadosesrcodcess: 
    if itemtesrcodcess not in entcod:
        print('ERRO!! ESCCODCESS não corresponde a um código de sociedade válido', itemtesrcodcess)
        critica7392_11 = ('Critica 7392.11, ESCCODCESS corresponde a um código de sociedade válido', itemtesrcodcess)
        criticas.append(critica7392_11)
    else:
        print('ENTCODIGO corresponde a um tipo de operação válida.', itemtesrcodcess)
        continue

In [ ]:
# Criando DataFrame copia com Campos CMPID, ESCCODCESS, ENTCODIGO no formato int
df_mains = df_main.copy()
df_mains['CMPID'] = df_mains['CMPID'].astype('int64')
df_mains['ESCCODCESS'] = df_mains['ESCCODCESS'].astype('int64')
df_mains['ENTCODIGO'] = df_mains['ENTCODIGO'].astype('int64')

In [ ]:
#7398.12 Valida a correspondência entre os campos CMPID e ESCCODCESS 
relacao12_65 = df_mains[(df_mains['CMPID'] == 1065) & (df_mains['ESCCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao12_69 = df_mains[(df_mains['CMPID'] == 1069) & (df_mains['ESCCODCESS'].values != df_mains['ENTCODIGO'].values)]
relacao12_73 = df_mains[(df_mains['CMPID'] == 1073) & (df_mains['ESCCODCESS'].values != df_mains['ENTCODIGO'].values)]

In [ ]:
i = 36   #Definindo numero de colunas

# Adicionando criticas com CMPIDs 1065, 1069, 1073 na lista 'criticas'
if relacao12_65.shape != (0,i): 
    critica7398_12_65 = ('Critica 7398.12, O CMPID é 1065 e o ESCCODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7398_12_65)
elif relacao12_69.shape != (0,i): 
    critica7398_12_69 = ('Critica 7398.12, O CMPID é 1069 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_69)
elif relacao12_73.shape != (0,i): 
    critica7398_12_73 = ('Critica 7398.12, O CMPID é 1073 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_73)
else: print('CMPIDs 1065, 1069, 1073. Valida campo ESCCODCESS')

In [ ]:
#7398.12 Valida a correspondência entre os campos CMPID e ESCCODCESS 
relacao12_66 = df_mains[(df_mains['CMPID'] == 1066) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 
relacao12_67 = df_mains[(df_mains['CMPID'] == 1067) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 
relacao12_70 = df_mains[(df_mains['CMPID'] == 1070) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 
relacao12_71 = df_mains[(df_mains['CMPID'] == 1071) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 
relacao12_74 = df_mains[(df_mains['CMPID'] == 1074) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 
relacao12_75 = df_mains[(df_mains['CMPID'] == 1075) & (df_mains['ESCCODCESS'].values <= 1) & (df_mains['ESCCODCESS'].values >= 9999)] 

In [ ]:
# Adicionando criticas com CMPIDs 1066, 1067, 1070, 1071, 1074, 1075 na lista 'criticas'
if relacao12_66.shape != (0,i): 
    critica7398_12_66 = ('Critica 7398.12, O CMPID é 1066 e o ESCCODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7398_12_66)
elif relacao12_67.shape != (0,i): 
    critica7398_12_67 = ('Critica 7398.12, O CMPID é 1067 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_67)
elif relacao12_70.shape != (0,i): 
    critica7398_12_70 = ('Critica 7398.12, O CMPID é 1070 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_70)
elif relacao12_71.shape != (0,i): 
    critica7398_12_71 = ('Critica 7398.12, O CMPID é 1071 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_71)
elif relacao12_74.shape != (0,i): 
    critica7398_12_74 = ('Critica 7398.12, O CMPID é 1074 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_74)
elif relacao12_75.shape != (0,i): 
    critica7398_12_75 = ('Critica 7398.12, O CMPID é 1075 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_75)
else: print('CMPIDs 1066, 1067, 1070, 1071, 1074, 1075. Valida campo ESCCODCESS')

In [ ]:
#7398.12 Valida a correspondência entre os campos CMPID e ESCCODCESS 
relacao12_68 = df_mains[(df_mains['CMPID'] == 1068) & (df_mains['ESCCODCESS'].values <= 30000) & (df_mains['ESCCODCESS'].values >= 59999)] 
relacao12_72 = df_mains[(df_mains['CMPID'] == 1072) & (df_mains['ESCCODCESS'].values <= 30000) & (df_mains['ESCCODCESS'].values >= 59999)]
relacao12_76 = df_mains[(df_mains['CMPID'] == 1076) & (df_mains['ESCCODCESS'].values <= 30000) & (df_mains['ESCCODCESS'].values >= 59999)]

In [ ]:
# Adicionando criticas com CMPIDs 1068, 1072, 1076 na lista 'criticas'
if relacao12_68.shape != (0,i): 
    critica7398_12_68 = ('Critica 7398.12, O CMPID é 1068 e o ESCCODCESS é diferente do ENTCODIGO') 
    criticas.append(critica7398_12_68)
elif relacao12_72.shape != (0,i): 
    critica7398_12_72 = ('Critica 7398.12, O CMPID é 1072 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_72)
elif relacao12_76.shape != (0,i): 
    critica7398_12_76 = ('Critica 7398.12, O CMPID é 1076 e o ESCCODCESS é diferente do ENTCODIGO')
    criticas.append(critica7398_12_73)
else: print('CMPIDs 1068, 1072, 1076. Valida campo ESCCODCESS')

In [ ]:
#7398.14 Verifica se os campos ESCDATAINICIORO, ESCDATAFIMRO, ESCDATAEMISSRO, ESCDATAINICIORD, ESCDATAFIMRD e ESCDATAEMISSRD 
#correspondem a uma data válida e verifica se estão entre os limites de trinta anos para mais ou menos em relação ao mês de referência. 

#Criando um DataFrame com colunas de Data para cpmparação. 
#Adicionando MRFMESANO minimo e maximo
df_dates_comparacao = pd.DataFrame(df_main['DTMRFMESANO'] - timedelta(days = 10950))
df_dates_comparacao.loc[:,'DTMRFMESANO_max'] = df_main['DTMRFMESANO'] + timedelta(days = 10950)

In [ ]:
mask14_i = (df_main['DTESCDATAINICIORO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAINICIORO'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_i = df_main.loc[mask14_i] 

if len(df14_i) > 0:
    df14_i.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_i

In [ ]:
mask14_f = (df_main['DTESCDATAFIMRO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAFIMRO'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_f = df_main.loc[mask14_f] 

if len(df14_f) > 0:
    df14_f.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_f

In [ ]:
mask14_e = (df_main['DTESCDATAEMISSRO'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAEMISSRO'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_e = df_main.loc[mask14_e] 

if len(df14_e) > 0:
    df14_e.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_e

In [ ]:
mask14_io = (df_main['DTESCDATAINICIORD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAINICIORD'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_io = df_main.loc[mask14_io] 

if len(df14_io) > 0:
    df14_io.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_io

In [ ]:
mask14_fr = (df_main['DTESCDATAFIMRD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAFIMRD'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_fr = df_main.loc[mask14_fr] 

if len(df14_fr) > 0:
    df14_fr.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_fr

In [ ]:
mask14_er = (df_main['DTESCDATAEMISSRD'] < df_dates_comparacao['DTMRFMESANO']) | (df_main['DTESCDATAEMISSRD'] > df_dates_comparacao['DTMRFMESANO_max'])
df14_er = df_main.loc[mask14_er] 

if len(df14_er) > 0:
    df14_er.loc[:,'ID da Critica'] = ('CRITICA 7398.14')
df14_er

In [ ]:
#7398.15 Verifica se a data do campo ESCDATAFIMRO é posterior à data do campo ESCDATAINICIORO e se a data do campo ESCDATAFIMRD 
#é posterior à data do campo ESCDATAINICIORD 
mask15_1 = df_main['DTESCDATAFIMRO'] < df_main['DTESCDATAINICIORO']
df15_1 = df_main.loc[mask15_1]
if len(df15_1) > 0:
    df15_1.loc[:,'ID da Critica'] = ('CRITICA 7398.15')
df15_1

In [ ]:
mask15_2 = df_main['DTESCDATAFIMRD'] < df_main['DTESCDATAINICIORD']
df15_2 = df_main.loc[mask15_2]
if len(df15_2) > 0:
    df15_2.loc[:,'ID da Critica'] = ('CRITICA 7398.15')
df15_2

In [ ]:
#7398.16 Verifica se o mês e ano da data do campo ESCDATAEMISSRO é igual ao mês e ano da data do campo MRFMESANO quando o tipo 
#de movimento for 'emissão' 
mask16_tpmoid = (df_main['TPMOID'] == '0007')
maskmr_16 = df_main['DTMRFMESANO'].dt.strftime("%Y-%m")
maskesr_16 = df_main['DTESCDATAEMISSRO'].dt.strftime("%Y-%m")

mask16 = ((maskmr_16 != maskesr_16)) & (mask16_tpmoid)
df16 = df_main.loc[mask16]
if len(df16) > 0:
    df16.loc[:,'ID da Critica'] = ('CRITICA 7398.16')
df16

In [ ]:
#7398.17 Verifica se os valores dos campos ESCDATAINICIORO, ESCDATAFIMRO, ESCDATAEMISSRO, ESCVALORMOVRO, ESCVALORCARO e ESCVALORCIRO 
#são iguais, respectivamente, aos valores dos campos ESCDATAINICIORD, ESCDATAFIMRD, ESCDATAEMISSRD, ESCVALORMOVRD, ESCVALORCARD 
#e ESCVALORCIRD quando o tipo de movimento for 'emissão' 
mask17_tpmoid = (df_main['TPMOID'] == '0007')
mask17_1 = (df_main['DTESCDATAINICIORO'] != df_main['DTESCDATAINICIORD'])|(df_main['DTESCDATAFIMRO'] != df_main['DTESCDATAFIMRD'])|(df_main['DTESCDATAEMISSRO'] != df_main['DTESCDATAEMISSRD'])
mask17_2 = (df_main['ESCVALORMOVRO'] != df_main['ESCVALORMOVRD'])|(df_main['ESCVALORCARO'] != df_main['ESCVALORCARD'])|(df_main['ESCVALORCIRO'] != df_main['ESCVALORCIRD'])
mask17_3 = (mask17_1)|(mask17_2)

mask17 = (mask17_tpmoid) & (mask17_3)
df17 = df_main.loc[mask17]
if len(df17) > 0:
    df17.loc[:,'ID da Critica'] = ('CRITICA 7398.17')
df17

In [ ]:
#7398.18 Se o tipo de movimento for 'emissão' e o valor dos campos ESCVALORCARO e ESCVALORCIRO for maior ou igual a zero, verifica se a soma dos campos ESCVALORCARO e ESCVALORCIRO é inferior ao valor do campo ESCVALORMOVRO

In [ ]:
#7398.19 Verifica se o mês e ano da data do campo ESCDATAEMISSRD é igual ao mês e ano da data do campo MRFMESANO quando o tipo 
#de movimento for 'aumento' ou 'restituição' ou 'cancelamento',
mask19_tpmoid = (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
maskmr_19 = df_main['DTMRFMESANO'].dt.strftime("%Y-%m")
maskesr_19 = df_main['DTESCDATAEMISSRD'].dt.strftime("%Y-%m")

mask19 = ((maskmr_19 != maskesr_19)) & (mask19_tpmoid)
df19 = df_main.loc[mask19]
if len(df19) > 0:
    df19.loc[:,'ID da Critica'] = ('CRITICA 7398.19')
df19

In [ ]:
#7398.20 Verifica se a data dos campos ESCDATAINICIORD e ESCDATAEMISSRD é igual ou posterior, respectivamente, à data dos campos
#ESCDATAINICIORO e ESCDATAEMISSRO quando o tipo de movimento for 'aumento' ou 'restituição' ou 'cancelamento' 
mask20_tpmoid = (df_main['TPMOID'] == '0008') | (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
mask20_1 = (df_main['DTESCDATAINICIORD'] < df_main['DTESCDATAINICIORO'])
mask20_2 = (df_main['DTESCDATAEMISSRD'] < df_main['DTESCDATAEMISSRO'])
mask20_3 = (mask20_1)|(mask20_2)

mask20 = (mask20_tpmoid) & (mask20_3)
df20 = df_main.loc[mask20]
if len(df20) > 0:
    df20.loc[:,'ID da Critica'] = ('CRITICA 7398.20')
df20

In [ ]:
#7398.21 Se o tipo de movimento for 'aumento' ou 'restituição' ou 'cancelamento'  e o valor dos campos ESCVALORCARD ou ESCVALORCIRD for maior ou igual a zero, verifica se a soma dos campos ESCVALORCARD e ESCVALORCIRD é inferior ao valor do campo ESCVALORMOVRD 

In [ ]:
#7398.22  Se o tipo de movimento for 'restituição' ou 'cancelamento', verifica se a data do campo ESCDATAEMISSRD é inferior a 
#90 dias após à data informada no campo ESCDATAFIMRD 
mask22_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
df_dates_comparacao.loc[:,'DTESCDATAFIMRD-90'] = df_main['DTESCDATAFIMRD'] - timedelta(days = 90)

mask22 = (mask22_tpmoid) & (df_main['DTESCDATAEMISSRD'] < df_dates_comparacao['DTESCDATAFIMRD-90'])
df22 = df_main[mask22]
if len(df22) > 0:
    df22.loc[:,'ID da Critica'] = ('CRITICA 7398.22')
df22

In [ ]:
#7398.23 Verifica se a data do campo ESCDATAFIMRD é igual à data do campo ESCDATAFIMRO quando o tipo de movimento for 
#'restituição' ou 'cancelamento'
mask23_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
mask23 = (mask23_tpmoid) & (df_main['DTESCDATAFIMRD'] != df_main['DTESCDATAFIMRO'])
df23 = df_main.loc[mask23]
if len(df23) > 0:
    df23.loc[:,'ID da Critica'] = ('CRITICA 7398.23')
df23

In [ ]:
#7398.24 Verifica se o valor dos campos ESCVALORMOVRD, ESCVALORCARD e ESCVALORCIRD é igual ou inferior, respectivamente, 
#ao valor dos campos ESCVALORMOVRO, ESCVALORCARO e ESCVALORCIRO quando o tipo de movimento for 'restituição' ou 'cancelamento', 
mask24_tpmoid = (df_main['TPMOID'] == '0009') | (df_main['TPMOID'] == '0010')
mask24_1 = (df_main['fESCVALORMOVRD'] != df_main['fESCVALORMOVRO'])|(df_main['fESCVALORCARD'] != df_main['fESCVALORCARO'])|(df_main['fESCVALORCIRD'] != df_main['fESCVALORCIRO'])

mask24 = (mask24_tpmoid) & (mask24_1)
df24 = df_main.loc[mask24]
if len(df24) > 0:
    df24.loc[:,'ID da Critica'] = ('CRITICA 7398.24')
df24

In [ ]:
#7398.25 Verifica se os campos ESCVALORCARO e ESCVALORCARD são iguais a zero  quando o tipo de operação for 'prêmio de resseguro' 
#(CMPID: 1068) ou 'restituição de prêmio de resseguro' (CMPID: 1072) ou 'cancelamento - repasse de resseguro' (CMPID: 1076) 
mask25_tpmoid = (df_main['CMPID'] == '1068') | (df_main['CMPID'] == '1072') | (df_main['CMPID'] == '1076')
mask25_1 = (df_main['fESCVALORCARO'] != 0.0)|(df_main['fESCVALORCARD'] != 0.0)

mask25 = (mask25_tpmoid) & (mask25_1)
df25 = df_main.loc[mask25]
if len(df25) > 0:
    df25.loc[:,'ID da Critica'] = ('CRITICA 7398.25')
df25

In [ ]:
#7398.26 Verifica se os campos ESCVALORCIRO e ESCVALORCIRD são iguais à zero quando o tipo de operação for 'cosseguro aceito' 
#(CMPID: 1066) ou 'cosseguro cedido' (CMPID: 1067) ou ' prêmio de resseguro' (CMPID: 1068) ou 'restituição de prêmio de cosseguro aceito' 
#(CMPID: 1070) ou 'restituição de prêmio de cosseguro cedido' (CMPID: 1071) ou 'restituição de prêmio de resseguro' (CMPID: 1072) 
#ou 'cancelamento total de prêmio de cosseguro aceito' (CMPID: 1074)ou ' cancelamento total de prêmio de cosseguro cedido' ou (CMPID: 1075)
#'cancelamento total de prêmio de resseguro' (CMPID: 1076)
mask26_tpmoid = (df_main['CMPID'] == '1066') | (df_main['CMPID'] == '1067') | (df_main['CMPID'] == '1068')|(df_main['CMPID'] == '1070') | (df_main['CMPID'] == '1071') | (df_main['CMPID'] == '1072')|(df_main['CMPID'] == '1074') | (df_main['CMPID'] == '1075') | (df_main['CMPID'] == '1076')
mask26_1 = (df_main['fESCVALORCIRO'] != 0.0)|(df_main['fESCVALORCIRD'] != 0.0)

mask26 = (mask26_tpmoid) & (mask26_1)
df26 = df_main.loc[mask26]
if len(df26) > 0:
    df26.loc[:,'ID da Critica'] = ('CRITICA 7398.26')
df26

In [ ]:
#7398.28 Verifica se os campos ESCVALORMOVRO E ESCVALORMOVRD são maiores que zero e se os campos ESCVALORCARO, ESCVALORCARD, 
#ESCVALORCIRO e ESCVALORCIRD são maiores ou iguais a zero 
mask28_1 = (df_main['fESCVALORMOVRO'] <= 0.0)|(df_main['fESCVALORMOVRD'] <= 0.0)
mask28_2 = (df_main['fESCVALORCARO'] < 0.0)|(df_main['fESCVALORCARD'] < 0.0)|(df_main['fESCVALORCIRO'] < 0.0)|(df_main['fESCVALORCIRD'] < 0.0)

mask28 = (mask28_1) | (mask28_2)
df28 = df_main.loc[mask28]
if len(df28) > 0:
    df28.loc[:,'ID da Critica'] = ('CRITICA 7398.28')
df28

In [ ]:
#7398.29  Para cada “PLNCODIGO” informado pela companhia no quadro estatístico 382 (exceto planos cadastrados com regime financeiro de capitalização), verifica se o resultado da soma do ESCVALORMOVRO ‘Emitido’(TPMOID: 0007; CMPID: 1065 + 1066 –1067) com a soma do campo ESCVALORMOVRD ‘Aumentado’(TPMOID: 0008; CMPID: 1065 + 1066 –1067), subtraído da soma do campo ESCVALORMOVRD ‘Restituído’(TPMOPID: 0009; CMPID: 1069 + 1070 –1071) e subtraído da soma do campo ESCVALORMOVRD ‘Cancelado’(TPMOID 0010; CMPID: 1073 + 1074 –1075) é igual ao campo ‘Valor do Prêmio/Contribuição Comercial’ nos Quadros 102 (CMPID: 12837) e 103 (CMPID: 12844). 

In [ ]:
#7398.30 Para cada “PLNCODIGO” informado pela companhia no quadro estatístico 382, verifica se o resultado da soma do campo ESCVALORCIRO ‘Emitido’ (TPMOID: 0007; CMPID: 1065) com a soma do campo ESCVALORCIRD ‘Aumentado’ (TPMOID: 0008; CMPID: 1065), subtraído da soma do campo ESCVALORCIRD ‘Restituído’ (TPMOID: 0009; CMPID: 1069) e subtraído da soma do campo ESCVALORCIRD ‘Cancelado’ (TPMOID: 0010; CMPID: 1073) é igual ao campo ‘Custos Iniciais’ nos Quadros 102 (CMPID: 12839) e 103 (CMPID: 12846) 

In [ ]:
#7398.31 Verifica se o campo ESCFREQ é um número inteiro positivo.
df_main.loc[:,'iESCFREQ'] = df_main['ESCFREQ'].astype('int64')

In [ ]:
#7398.32 Para os registros cujo PLNCODIGO informado se refira a um plano de previdência tradicional, verifica se o valor do campo ESCIMPSEG dividido pelo valor do campo ESCFREQ é menor ou igual ao Limite de Retenção informado no cadastro de planos de previdência tradicionais (CMPID 13108). 

In [ ]:
#7398.33 Se o MRFMESANO for maior ou igual a 20180131 e o tipo de movimento for “Emissão de Prêmio”, verificar se o campo 
#ESCVALORCIRO é igual a zero.
mask33_1 = (df_main['MRFMESANO'] >= '20180131') & (df_main['TPMOID'] == '0007')
mask33 = (mask33_1) & (df_main['fESCVALORCIRO'] != 0.0)
df33 = df_main.loc[mask33]
if len(df33) > 0:
    df33.loc[:,'ID da Critica'] = ('CRITICA 7398.33')
df33

In [ ]:
#7398.34 Se o MRFMESANO for maior ou igual a 20180131 e o tipo de movimento for “Aumento de Prêmio”, verificar se o campo 
#ESCVALORCIRD é igual a zero.
mask34_1 = (df_main['MRFMESANO'] >= '20180131') & (df_main['TPMOID'] == '0008')
mask34 = (mask34_1) & (df_main['fESCVALORCIRD'] != 0.0)
df34 = df_main.loc[mask34]
if len(df34) > 0:
    df34.loc[:,'ID da Critica'] = ('CRITICA 7398.34')
df34

In [ ]:
################################################
# Adicionando Nome do Criador do output e data #
################################################

df_main.loc[:,'Criador'] = input("Insira seu nome: ") ### insere a coluna com o nome do criador da tabela
data_presente = datetime.now()
df_main.loc[:,'Data do output'] = data_presente ### insere a coluna com a data de criacao da tabela

In [ ]:
df_main

In [ ]:
criticas

In [ ]:
df_criticas = pd.DataFrame(criticas)
df_criticas = df_criticas.rename(columns={0:'Descrição'})
df_criticas

In [ ]:
df_criticas_2 = pd.concat([df14_i,df14_f,df14_e,df14_io,df14_fr,df14_er,df15_1,df15_2,df16,df17,df19,df20,df22,df23,df24,df25,df26,df28,df33,df34])
df_criticas_2

In [ ]:
if criticas == []:                   # Verificando se a lista de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 382')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 382')
    df_criticas.to_csv('Criticas_nao_impeditivas382.csv')     # Gerando csv das criticas impeditivas

In [ ]:
if len(df_criticas_2) == 0:                             # Verificando se o DataFrame de criticas esta vazia
    print('Nenhuma Critica NÃO Impeditiva encontrada no Quadro 382')
else:
    print('Foi encontrada uma ou mais Criticas NÃO Impeditivas no Quadro 382')
    df_criticas_2.to_csv('Criticas_nao_impeditivas382_2.csv')         # Gerando csv das criticas impeditivas

In [ ]:
df_main.to_csv('382_Trabalho.csv')